In [1]:
!pip install -q streamlit

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_path = "/content/drive/MyDrive/Lighthouse Labs/lhl-final-project"
import pandas as pd

In [4]:
#

In [5]:
app_code = """
import streamlit as st
import pandas as pd

# Load your final data
df = pd.read_csv("/content/drive/MyDrive/Lighthouse Labs/lhl-final-project/data/final_df.csv")

# Smart category mapping from product name
category_keywords = {
    "Serums": ["serum"],
    "Vegan": ["vegan"],
    "Moisturizers": ["moisturizer", "cream", "lotion"],
    "Treatments": ["treatment", "spot", "booster"],
    "Masks": ["mask", "sleeping mask", "clay"],
    "Lip Balms & Treatments": ["lip", "lip balm", "lip treatment"],
    "Cleansers": ["cleanser", "face wash", "foam"],
    "Value & Gift Sets": ["set", "gift", "Trial Kit", "kit"],
    "Eye Care": ["eye", "eye cream", "dark circle"],
    "Sunscreen": ["spf", "sunscreen", "uv"],
    "Mini Size": ["mini", "travel", "sample"]
}

def detect_category(product_name):
    name = product_name.lower() if pd.notnull(product_name) else ""
    matches = [label for label, keywords in category_keywords.items() if any(k in name for k in keywords)]
    return matches[0] if matches else "Other"

df['category_label'] = df['product_name'].apply(detect_category)

# Sidebar – Skin Type & Filters
st.sidebar.markdown("## Customize Your Skin Search")
skin_types = ["dry", "oily", "sensitive", "acne-prone", "mature", "dull"]
skin_type = st.sidebar.selectbox("Select your skin type", skin_types)

# Category dropdown
available_categories = sorted(df['category_label'].unique())
selected_category = st.sidebar.selectbox("Filter by category (optional)", ["All"] + available_categories)

# Clean numeric columns
df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
df['price_usd'] = pd.to_numeric(df['price_usd'], errors='coerce')

# Price range filter
min_price = float(df['price_usd'].min())
max_price = float(df['price_usd'].max())
price_range = st.sidebar.slider("Select your price range", min_price, max_price, (min_price, max_price))

# Toggle: only affordable products
show_affordable_only = st.sidebar.checkbox("under $20")

# Title + Subtitle
st.markdown("<h1 style='text-align: center; color: hotpink;'>Better Skin</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center; color: deeppink;'>✨ Your glow-up starts with the right routine ✨</p>", unsafe_allow_html=True)

# Search bar
search_query = st.text_input("Search for a product or brand", "")

# Filter logic
if selected_category != "All":
    df_filtered = df[df['category_label'] == selected_category]
else:
    df_filtered = df[df['category_label'] != "Value & Gift Sets"].copy()

df_filtered = df_filtered[(df_filtered['price_usd'] >= price_range[0]) & (df_filtered['price_usd'] <= price_range[1])]

if show_affordable_only:
    df_filtered = df_filtered[df_filtered['price_usd'] < 20]

if search_query:
    df_filtered = df_filtered[
        df_filtered['product_name'].str.contains(search_query, case=False, na=False) |
        df_filtered['brand_name'].str.contains(search_query, case=False, na=False)
    ]

# Filter by relevance, then sort top 50 by rating
score_col = f'relevance_score_{skin_type}'
df_filtered[score_col] = pd.to_numeric(df_filtered[score_col], errors='coerce')
df_filtered['rating'] = pd.to_numeric(df_filtered['rating'], errors='coerce')
df_top_relevant = df_filtered.sort_values(by=score_col, ascending=False).head(50)
df_sorted = df_top_relevant.sort_values(by='rating', ascending=False)

# Display results
st.markdown(f"#### 🎀 Best Products for **{skin_type}** Skin (Top 50 by relevance, sorted by rating):")

for idx, row in df_sorted.iterrows():
    product = row['product_name']
    brand = row['brand_name']
    price = row['price_usd']
    rating = row['rating']
    category = row['category_label']

    sephora_url = f"https://www.sephora.com/search?keyword={product.replace(' ', '+')}+{brand.replace(' ', '+')}"

    st.markdown(f'''
    <div style="border: 1px solid #f9c1d9; padding: 15px; border-radius: 15px; margin-bottom: 10px; background-color: #fff0f5;">
        <h4 style="color: hotpink;">{product}</h4>
        <p style="color: #8e7cc3;">
            <b>Brand:</b> {brand} &nbsp;|&nbsp;
            <b>Price:</b> ${price:.2f} &nbsp;|&nbsp;
            <b>Rating:</b> {rating:.1f}
        </p>
        <p style="color: #8e7cc3; margin-top: -10px;"><i>Category:</i> {category}</p>
        <a href="{sephora_url}" target="_blank" style="color:white; background-color: hotpink; padding: 6px 12px; text-decoration: none; border-radius: 6px;">🔗 View on Sephora</a>
    </div>
    ''', unsafe_allow_html=True)

# Footer
st.markdown("<p style='text-align: center; font-size: 12px; color: palevioletred;'>Made with 💗 by Diba Jafarnejad</p>", unsafe_allow_html=True)
"""

In [6]:
with open("app.py", "w") as f:
    f.write(app_code)

In [7]:
!npm install -g localtunnel --yes

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
changed 22 packages in 4s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [8]:
!pip install -q streamlit pyngrok


In [9]:
from pyngrok import ngrok
ngrok.set_auth_token("2w1HihenztImNlAsreIOVJuuRxd_tw2oNA17auvHw1XSyKrb")


In [10]:
#from pyngrok import ngrok
#ngrok.kill()

In [11]:
import subprocess
import time

# Launch Streamlit in background
process = subprocess.Popen(["streamlit", "run", "app.py"])
time.sleep(5)

In [12]:
# Open ngrok tunnel
public_url = ngrok.connect(addr="8501", proto="http")
print("🌍 App is live at:", public_url)

🌍 App is live at: NgrokTunnel: "https://cd9a-35-185-49-78.ngrok-free.app" -> "http://localhost:8501"
